In [1]:
import numpy as np
import cv2

cap = cv2.VideoCapture('input_video.mp4')

# List to store previous centeroid position
cent_prev = []

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('OUTPUT.avi',fourcc, 25.0, (640,720))

# Function to Identify, Draw circle and label centeroids
def identify_centeroid(image, contour,centeroids):
    cent_moment = cv2.moments(contour)
    cent_x = int(cent_moment['m10'] / cent_moment['m00'])
    cent_y = int(cent_moment['m01'] / cent_moment['m00'])
    label = str(cent_x) + "," + str(cent_y)
    cv2.circle(image,(cent_x,cent_y), 5, (255,255,255), -1)
    cv2.putText(image, label, (cent_x-50, cent_y-20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255,255,255), 2);
    centeroids.append((cent_x-70,cent_y-50))
    return image

# Function to return X coordinate of centeroid
def ret_x_cont(contour):
    cent_moment = cv2.moments(contour)
    cent_x = int(cent_moment['m10'] / cent_moment['m00'])
    return cent_x

In [2]:
while cap.isOpened():
    ret, frame = cap.read()
    if ret == 0:
        break
    # Creating a black image with similar dimensions to frame
    res = np.zeros_like(frame)
    
    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # define range of red color in HSV
    lower_red = np.array([65,0,0])
    upper_red = np.array([179,255,255])

    # Threshold the HSV image to get only red colors
    mask = cv2.inRange(hsv, lower_red, upper_red)
    
    # Apply blurr to the mask
    blurr = cv2.GaussianBlur(mask,(15,15),0)
    
    # Finding contours contours of the blurred image
    contours1, hierarchy = cv2.findContours(blurr, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Sorting contours by X coordinate of centeroid
    contours = sorted(contours1, key=ret_x_cont, reverse=False)
    
    # List to store centeroid position
    centeroids = []
    
    # Approximating contours and identifying centeroids
    for cont in contours:
        accuracy = 0.002 * cv2.arcLength(cont, True)
        approx = cv2.approxPolyDP(cont, accuracy, True)
        identify_centeroid(res, approx, centeroids)
    
    if int(cap.get(0))>1:
    # Drawomg arrowed lines in the deirectoin of movement
        for i in range(len(centeroids)):
            p1 = (cent_prev[i])
            p2 = (centeroids[i])
            theta = np.arctan2(p1[1]-p2[1], p1[0]-p2[0])
            endpt_x = int(p1[0] - abs(p2[0]-p1[0])*10*np.cos(theta))
            endpt_y = int(p1[1] - abs(p2[1]-p1[1])*10*np.sin(theta))
            cv2.arrowedLine(res, (cent_prev[i]),  (endpt_x, endpt_y), (255,255,255), 2);
    
    cent_prev = centeroids
    
    out.write(res)
    cv2.imshow('OUTPUT',res)
    cv2.imshow('INPUT',frame)
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

In [3]:
cap.release()
out.release()
cv2.destroyAllWindows()